In [8]:
# load 4-class data
X = np.load("features_4class.npy")
print(np.shape(X))

(906, 10, 862)


In [9]:
# PCA 
from sklearn.decomposition import PCA
proj = np.zeros(shape=(906,10,862))

for i in range(906):
    X1 = X[i,:,:]
    pca = PCA(whiten=True)
    X_pca = pca.fit_transform(X1)
    proj[i,:,:] = pca.inverse_transform(X_pca)

X = proj

In [10]:
y_accent = datanew['native_language']

encoder1 = LabelEncoder()
encoder1.fit(y_accent)
y_accent_ = encoder1.transform(y_accent)
y_accent_ = to_categorical(np.array(y_accent_),dtype='float32')
print(np.shape(y_accent_))  #4 categories.

(906, 4)


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_accent_train, y_accent_test = train_test_split(X, y_accent_, test_size=0.2, random_state=10)

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std
X_train= np.expand_dims(X_train, axis=3)
X_test= np.expand_dims(X_test, axis=3)

#X_train, X_val, y_accent_train, y_accent_val = train_test_split(X_train, y_accent_train, test_size=0.15, random_state=10)
#X_val = (X_val - mean)/std
#X_val= np.expand_dims(X_val, axis=3)

print ('Train set:', X_train.shape,  y_accent_train.shape)
print ('Test set:', X_test.shape,  y_accent_test.shape)
#print ('validation set:', X_val.shape,  y_accent_val.shape)
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std
X_train= np.expand_dims(X_train, axis=3)
X_test= np.expand_dims(X_test, axis=3)

#X_train, X_val, y_accent_train, y_accent_val = train_test_split(X_train, y_accent_train, test_size=0.15, random_state=10)
#X_val = (X_val - mean)/std
#X_val= np.expand_dims(X_val, axis=3)

print ('Train set:', X_train.shape,  y_accent_train.shape)
print ('Test set:', X_test.shape,  y_accent_test.shape)
#print ('validation set:', X_val.shape,  y_accent_val.shape)
print(y_accent_train)

Train set: (724, 10, 862, 1) (724, 4)
Test set: (182, 10, 862, 1) (182, 4)
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


# SVM

In [85]:
y_accent_ml = np.zeros(shape=(906,1))
X_ml = np.reshape(X,(906,8620))
for i in range(906):
    for j in range(4):
        if y_accent_[i,j]==1:
            y_accent_ml[i,0] = j
            
from sklearn.model_selection import train_test_split

X_train_ml, X_test_ml, y_accent_train_ml, y_accent_testml = train_test_split(X_ml, y_accent_ml, test_size=0.2, random_state=10)
print(X_train_ml.shape, y_accent_train_ml.shape)

(724, 8620) (724, 1)


In [86]:
# reshape data
y_accent_train_ml = np.zeros(shape=(724,1))
X_train_ml = np.reshape(X_train,(724,8620))
print(X_train_ml.shape)
y_accent_test_ml = np.zeros(shape=(182,1))
X_test_ml = np.reshape(X_test,(182,8620))

for i in range(724):
    for j in range(4):
        if y_accent_train[i,j]==1:
            y_accent_train_ml[i,0] = j
for i in range(182):
    for j in range(4):
        if y_accent_test[i,j]==1:
            y_accent_test_ml[i,0] = j
print(y_accent_train_ml.shape)

(724, 8620)
(724, 1)


In [88]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score as acc

svm = SVC(kernel="rbf")
print("svc train:", acc(y_accent_train_ml,svc.fit(X_train_ml,y_accent_train_ml).predict(X_train_ml)))
print("svc test:", acc(y_accent_test_ml,svc.fit(X_test_ml,y_accent_test_ml).predict(X_test_ml)))


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc train: 0.9019337016574586


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc test: 0.8186813186813187


In [71]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0,max_depth=3)
print("dt train:", acc(y_accent_train_ml,dt.fit(X_train_ml, y_accent_train_ml).predict(X_train_ml)))
print("dt test:", acc(y_accent_test_ml,dt.fit(X_train_ml, y_accent_train_ml).predict(X_test_ml)))

dt: 0.6740331491712708
dt: 0.6208791208791209


In [92]:
y_accent_ml = np.zeros(shape=(906,1))
X_ml = np.reshape(X,(906,8620))
for i in range(906):
    for j in range(4):
        if y_accent_[i,j]==1:
            y_accent_ml[i,0] = j

from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5, shuffle=False) 

for k, (train, test) in enumerate(k_fold.split(X_ml)):
    print("svc train:", acc(y_accent_ml[train],svm.fit(X_ml[train],y_accent_ml[train]).predict(X_ml[train])))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc train: 0.5538674033149171


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc train: 0.56


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc train: 0.5972413793103448


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc train: 0.7531034482758621


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


svc train: 0.7986206896551724
